# 실습

In [ ]:
'''
레벨 : 초급
스타일 : 밸류+퀄리티
기대CAGR : 15% ~ 20%
종목개수 : 20~30
매수전략 : ROA 5% 이상 , 부채비율 50% 이하 , PBR 낮은 기업부터 ( PBR <0.2 제외)
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 스냅샷 URL
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
# 재무비율 URL
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

In [ ]:
sample_code = '005930'

In [ ]:
snap_url = SNAP_URL % sample_code # SNAP_URL에 formatting 한 것을 snap_url에 저장
snap_content = requests.get(snap_url).content # snap_url을 가져옴
snap_tree = html.fromstring(snap_content) #  

In [ ]:
pbr = snap_tree.xpath('//*[@id="corp_group2"]/dl[4]/dd')[0].text
pbr = float(pbr)
pbr

In [ ]:
# 부채비율 크롤링
ratio_url = RATIO_URL % sample_code
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)

In [ ]:
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)
debt_ratio


In [ ]:
roa = ratio_tree.xpath('//*[@id="p_grid1_17"]/td[5]')[0].text
roa = float(roa)
roa

In [ ]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)
print(company.shape)
company.head(10)

In [ ]:
# company.종목코드
code_list = company['종목코드'] 
code_list

In [ ]:
sample_code_list = code_list[1000:1050]
sample_code_list

In [ ]:
def crawler(li) :
    
     # fomatting이 비어있는 snap_url, ratio_url을 만들어야 함
    SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
    RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
    result = {} # result라는 비어있는 딕셔너리를 만듬
    for code in tqdm(li) : 
        try:
            #비어있는 딕셔너리 하나 만듬
            
            #get pbr
            snap_url = SNAP_URL % code 
            snap_content = requests.get(snap_url).content 
            snap_tree = html.fromstring(snap_content) 

            pbr = snap_tree.xpath('//*[@id="corp_group2"]/dl[4]/dd')[0].text
            pbr = float(pbr)

            #get debt ratio
            ratio_url = RATIO_URL % code
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)

            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)

            #get roa
            roa = ratio_tree.xpath('//*[@id="p_grid1_17"]/td[5]')[0].text
            roa = float(roa)

            #company name
            name = company[company['종목코드'] == code]['회사명'].values[0] 
            result[name] = [code, pbr, debt_ratio, roa] #list안에 value 값

        except(TypeError, AttributeError, IndexError, ValueError) as err :
            # 에러가 났을 때 무시하고 다음으로 넘어감
            pass 
    result_df = pd.DataFrame(result)
    #전치 행렬

    result_df = result_df.transpose()

    result_df.columns = ['Code', 'PBR', 'Dedt_ratio', 'ROA']

    return result_df

In [ ]:
result_df = crawler(sample_code_list)
result_df

In [ ]:
# copy 메서드
copy_df = result_df.copy()
copy_df

In [ ]:
filtered_df = copy_df[(copy_df['PBR'] >= 0.2) & (copy_df['ROA'] >= 5) & (copy_df['Dedt_ratio'] <= 50)]
filtered_df

In [ ]:
filtered_df.sort_values(by='PBR').head(30)